In [ ]:
'''
y is the sequence of tags while x is the observed sequence of words 
Code dies when doing CN and SG files.
'''

In [83]:
'''
General functions
read & write requires encoding='utf8' due to different languages in the files.
'''
#read the training data file 
def read_file_train(fp):
    print("reading train file...\n")
    parsed = []
    f = open(fp,'r',encoding='utf8')
    while True:
        line = f.readline() 
        #clean up the data read 
        if line == '\n':
            parsed.append(line.replace('\n','Blank Blank')) #Replace spaces with Blanks.
        elif line == "":
            break
        else:
            parsed.append(line.strip()) #remove \n at the end 
        
    return parsed

#read the test data file 
def read_file_test(fp):
    print("reading tests file...\n")
    parsed = []
    f = open(fp,'r',encoding='utf8')
    while True:
        line = f.readline() 
        #clean up the data read 
        if line == '\n':
            parsed.append(line.replace('\n','Blank')) #Replace spaces with Blanks.
        elif line == "":
            break
        else:
            parsed.append(line.strip()) #remove \n at the end 
        
    return parsed

'''
data received shall be in dictionary term of 
data = ((x1 ,y),(x2,y),...)
fn = filename 
fp = filepath
'''
def write_file(fp,fn,data):
    print("writing file...\n")
    with open(fp+fn, 'w',encoding='utf8') as f:
        for k in data:
#             print("data received:")
#             print(k[0] + ' ' + k[1]+'\n')
            if k[0] == "Blank": #replace the blanks
                f.write("\n")
            else:
                f.write(k[0] + ' ' + k[1]+'\n')
    f.close()
    

# build dictionaries of count(y->x) and count(y)
def built_dict(train_data):
    print("building count(y->x) & count(y)...\n")
    '''
    data will be a array of the read train data 
    1. create tuples:count out of the y and x and put them in global_y_x
    2. if the tuples were repeated, don't add a new entry into the dictionary but increase the count 
 
    1. from global_y_x, we have gotten count(y-> x) . 
    2. need get count(y)
    
    returns huge_count_y_x and huge_count_y 
    huge_count_y_x = {("We","B-Positive"):1}
    huge_count_y = {"B-Positive":1}
    '''
    
    huge_count_y_x = {} #dict to contain all count(y->x)
    huge_count_y = {} #dict to contain all count(y)
    for d in train_data:
        x,y = d.split(" ") #split by spaces 
        temp_tuple = (x,y)
        
        if temp_tuple not in huge_count_y_x:
            huge_count_y_x[temp_tuple] = 1
        else:
            huge_count_y_x[temp_tuple] = huge_count_y_x[temp_tuple] + 1 
        
        if y not in huge_count_y:
            huge_count_y[y] =  1
        else: 
            huge_count_y[y] = huge_count_y[y] + 1
    return huge_count_y_x,huge_count_y 

#part 2 first part 

def calculate_emmision(huge_count_y_x, huge_count_y):
    print("calculating emission probability...\n")
    '''
    Calculating the emission 
    Use the tuple (x,y) to represent the occurance of e(x|y)  
    '''
    huge_emission ={} #dict to contain all the emission probabilities 
    for seq, count_y_x in huge_count_y_x.items():
        
        emission = 0.0 #if none found, then return as 0 
        tag = seq[1]
#         print(k,v)
        count_y = huge_count_y[tag] 
        emission = count_y_x / count_y
        huge_emission[seq] = emission

    return huge_emission

# parsed_train = read_file_train("EN/train")
# print(calculate_emmision(parsed_train))

#part 2 second part 
'''
    replace words that appear < k times with #UNK# 
    modify_train_set accepts k, training data
    training data is a list returned from read_file_train
'''
def modify_train_set(k,train_data):
    print("modifying to #UNK#...\n")
    modified_training_data = []
    count_dict = {}
    for td in train_data:
#         print(td)
        word,tag = td.rsplit(" ",1) 
        if word not in count_dict:
            count_dict[word] = 1
        else:
            count_dict[word] = count_dict[word] + 1
    #identify which words appeared less than k times 
    list_of_works_less_than_k = [] 
    for w,count in count_dict.items():
        if count < k and w != "Blank": #Skip blanks. 
            list_of_works_less_than_k.append(w)
    #now replace the entries in the training set  
    for td in train_data:
        word,tag = td.rsplit(" ",1)
        for words_to_be_removed in list_of_works_less_than_k:
            if word == words_to_be_removed:
                index = train_data.index(td)
                train_data[index] = '#UNK# ' + tag
    return train_data
                
#part 2 third part 
'''
using the emission data, predict the possible tags (?) 
the tuple (x,y) is key to occurance of e(x|y) 
TODO:
1. replace with #UNK# 
2. for each word w in the test_data
    3. find the tuples in huge_emissison that matches w
    4. if there's more than one, select the one with max(emission probability)
    5. if there's only 1 result, then use that tag 
    6. append the tag to y_star
7. write the results of w, tag into a tuple
8. return the array y_star of tuples

y_star = [("We","O"),("are","I"),...]
'''
def simple_sentiment_analysis(test_data,modified_train_data,huge_emission):
    print("doing simple_sentiment_analysis...\n")
    #extract the words 
    extracted_train_words = []
    for d in modified_train_data:
        x,y = d.rsplit(" ",1)
        extracted_train_words.append(x)
    #check whether the word exists in train. else replace with #UNK# 
    for w in test_data:
        if w not in extracted_train_words and w != "Blank": #Avoid blanks.
            index = test_data.index(w)
            test_data[index] = '#UNK#'
    y_star=[] #the tags, to be appended here 
    for w in test_data:
        temp_dict = {} # to hold the possible results found from huge_emission 
        for key,e_value in huge_emission.items():
            #TODO: FIX THE SPACE MESS if it matches the word
            if w == key[0]: 
                temp_dict[key] = e_value
        #now return the tag that has the max e probability

        best_result = max(temp_dict, key=temp_dict.get)[1]

        #append a tupple for word, tag

        y_star.append((w,best_result))
    #return the dictionary -- don't do dic cos will remove duplicates 

    return y_star

#part 3 first part 
'''
first modify the train_data to get START and STOP in the data.
train data = ['We A','are B','young C','Blank Blank','Hello D'] from read_file_train(fp+"train")
test data = ['We','are','young','Blank','Hello'] from read_file_test(fp+"dev.in")
Change to 
train data = ['START','We A','are B','young C','STOP','START','Hello D','STOP'] 
test data = ['START','We','are','young','STOP','START','Hello','STOP']
'''
def mod_data_for_transition(td,data_type):
    print("Inserting START and STOP for " + data_type +"...\n")
    wtd = [] #words from train_data
    if data_type == "train":
        #extract the words first 
        for d in td:
            w,t = d.rsplit(" ",1) 
            wtd.append(w)
    else:
        wtd = td #don't need to extract 
    
    #start replacing 
    for i in range(len(wtd)):
        if wtd[i] == 'Blank':
            index = i
            wtd[index] = "STOP"
            wtd.insert(index+1,"START")
    wtd.insert(0,"START")
    wtd[len(wtd)-1]="STOP"
    #TODO: am not sure why we need to do it twice. 
    for w in wtd:
        if w == "Blank":
            index = wtd.index(w)
            wtd[index] = "STOP"
            wtd.insert(index+1,"START")
    return wtd 
    
    
    
def build_dict_transition(td): 
    print("constructing dictionaries for transition...\n")
    '''
    td=[strings]
    1. build a tuple for (yi,yi-1)
    2. put in dict 
    huge_count_yi_1_yi = {(yi,yi-1):1,...}
    3. extract key[1] and put in huge_count_yi_1 
    huge_count_yi_1 = {"yi-1":1,...}
    '''
    huge_count_yi_1_yi = {} #stores the no. of times of transition 
    huge_count_yi_1 = {} #stores no of times yi-1 occured 
#     print(len(td))
    for i in range(1,len(td)):
        # i = 0,1,2,...,n 
        temp = (td[i-1],td[i])
        if temp not in huge_count_yi_1_yi:
            huge_count_yi_1_yi[temp] = 1
        else:
            huge_count_yi_1_yi[temp] = huge_count_yi_1_yi[temp] + 1 
#         print (temp)
        if td[i-1] not in huge_count_yi_1:
            huge_count_yi_1[td[i-1]] = 1
        else:
            huge_count_yi_1[td[i-1]] = huge_count_yi_1[td[i-1]] + 1
            
    return huge_count_yi_1_yi,huge_count_yi_1

def calculate_transition(huge_count_yi_1_yi,huge_count_yi_1):
    print("calculating transition probabilities...\n")
    '''
    huge_transition = {(yi_1,yi):1,...}
    '''
    huge_transition = {}
    for yi_1_yi,count_yi_1_yi in huge_count_yi_1_yi.items():
        transition = 0.0 #ensure float
        for yi_1,count_yi_1 in huge_count_yi_1.items(): 
            if yi_1 == yi_1_yi[0]:
                transition = float(count_yi_1_yi / count_yi_1) #ensure float 
                huge_transition[yi_1_yi] = transition
    return huge_transition

#nightmare part 3 second part: viterbi algo. 
def viterbi(emissison,transition):
    y_star = [] #contains tuples of word and tags 
    return y_star

In [84]:
#code to run the file 
PART2_FN = "dev.p2.out"
PART3_FN = "dev.p3.out"
PART4_FN = "dev.p4.out"
EN_FP = "EN/"
FR_FP = "FR/"
CN_FP = "CN/"
SG_FP = "SG/"
FPList = [EN_FP,FR_FP,CN_FP,SG_FP]

In [85]:
#part 3 
print("Part 3 running...\n")
for fp in FPList:
    print("\nDoing " + fp + "...")
    parsedtrainData = read_file_train(fp+"train")
    # print(parsedtrainData)
    modifiedData = modify_train_set(3,parsedtrainData)
    stsp_train = (mod_data_for_transition(modifiedData,"train"))
    huge_count_yi_1_yi,huge_count_yi_1 = build_dict_transition(stsp_train)
    huge_transition = calculate_transition(huge_count_yi_1_yi,huge_count_yi_1)
    # print(huge_transition)
    parsedtestData = read_file_test(fp+"dev.in")
    stsp_test = mod_data_for_transition(parsedtestData,"test")
    print("End...\n")


Doing EN/...
reading train file...

modifying to #UNK#...

Inserting START and STOP for train...

constructing dictionaries for transition...

calculating transition probabilities...

reading tests file...

Inserting START and STOP for test...


Doing FR/...
reading train file...

modifying to #UNK#...

Inserting START and STOP for train...

constructing dictionaries for transition...

calculating transition probabilities...

reading tests file...

Inserting START and STOP for test...


Doing CN/...
reading train file...

modifying to #UNK#...

Inserting START and STOP for train...

constructing dictionaries for transition...

calculating transition probabilities...

reading tests file...

Inserting START and STOP for test...


Doing SG/...
reading train file...

modifying to #UNK#...

Inserting START and STOP for train...

constructing dictionaries for transition...

calculating transition probabilities...

reading tests file...

Inserting START and STOP for test...



In [86]:
#part 2
print("Part 2 running...\n")
for fp in FPList:
    print("\nDoing " + fp + "...")
    parsedtrainData = read_file_train(fp+"train")
    # print(parsedtrainData)
    modifiedData = modify_train_set(3,parsedtrainData)
    # print(modifiedData)
    huge_count_y_x, huge_count_y = built_dict(modifiedData)
    # print(huge_count_y_x)
    huge_emission = calculate_emmision(huge_count_y_x, huge_count_y)
    # # print(huge_emission)
    parsedtestData = read_file_test(fp+"dev.in")
    # print(parsedtestData)
    y_star = simple_sentiment_analysis(parsedtestData,modifiedData,huge_emission)
    # print(y_star)
    write_file(fp,PART2_FN,y_star)
    print("End...")


Doing EN/...
reading train file...

modifying to #UNK#...

building count(y->x) & count(y)...

calculating emission probability...

reading tests file...

doing simple_sentiment_analysis...

writing file...

End...

Doing FR/...
reading train file...

modifying to #UNK#...

building count(y->x) & count(y)...

calculating emission probability...

reading tests file...

doing simple_sentiment_analysis...

writing file...

End...

Doing CN/...
reading train file...

modifying to #UNK#...

building count(y->x) & count(y)...

calculating emission probability...

reading tests file...

doing simple_sentiment_analysis...

writing file...

End...

Doing SG/...
reading train file...

modifying to #UNK#...

building count(y->x) & count(y)...

calculating emission probability...

reading tests file...

doing simple_sentiment_analysis...

writing file...

End...
